In [80]:
BENCHMARK_TOOL = 'NVPROF' # or 'NVPROF'

import subprocess
import numpy as np
import pandas as pd
import io
from typing import Tuple

In [81]:
def read_nvprof(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[3].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

    
def read_ncu(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[2].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

def benchmark_kernel(exe_path: str, *args) -> str :
    if BENCHMARK_TOOL == 'NVPROF':
        return str(subprocess.check_output(
            ['nvprof', '--print-gpu-trace', '--csv', exe_path, *args], 
            stderr=subprocess.STDOUT
        ))
    elif BENCHMARK_TOOL == 'NCU':
        return str(subprocess.check_output(
            ['ncu','--csv', exe_path, *args], 
            stderr=subprocess.STDOUT, shell=True
        ))

def read_benchmark(output: str) -> pd.DataFrame:
    if BENCHMARK_TOOL == 'NVPROF':
        return read_nvprof(output)
    elif BENCHMARK_TOOL == 'NCU':
        return read_ncu(output)

def get_kernel_duration(df: pd.DataFrame) -> float:
    if BENCHMARK_TOOL == 'NVPROF':
        return float(df.loc[2, 'Duration']) * 1e-3
    elif BENCHMARK_TOOL == 'NCU':
        return float(df[df['Metric Name'] == 'Duration']['Metric Value'].to_numpy().item().replace(',','')) * 1e-6

In [90]:
get_kernel_duration(read_benchmark(benchmark_kernel("./gen", "-n", str(10))))

0.011584

In [94]:
arr = []
i = 1
while i <= 1024:
    arr.append(get_kernel_duration(read_benchmark(benchmark_kernel("./bench", "-f", "./img/chateau.png", "-n", str(i)))))
    i *= 2
print(arr)


[0.023264, 0.016768, 0.013216, 0.011808, 0.011136, 0.011359000000000001, 0.011104, 0.011328, 0.011231999999999999, 0.010432, 0.01056]


Meilleurs valeur de n:

In [101]:
2**np.argmin(arr)

512